In [ ]:
#Pieces detection
SRC_FOLDER = 'dataset/train'
DST_IMAGES_FOLDER = 'dataset/images/train'
DST_LABELS_FOLDER = 'dataset/labels/train'
N = 1000  

import os
import shutil
import json
from PIL import Image


#get all files in the source folder
files = os.listdir(SRC_FOLDER)

# empty the destination folders if they exist
if os.path.exists(DST_IMAGES_FOLDER):
    shutil.rmtree(DST_IMAGES_FOLDER)
if os.path.exists(DST_LABELS_FOLDER):
    shutil.rmtree(DST_LABELS_FOLDER)
# create destination folders
os.makedirs(DST_IMAGES_FOLDER, exist_ok=True)
os.makedirs(DST_LABELS_FOLDER, exist_ok=True)

classes = {
    'B': 0, 'K': 1, 'N': 2, 'P': 3, 'Q': 4, 'R': 5,
    'b': 6, 'k': 7, 'n': 8, 'p': 9, 'q': 10, 'r': 11
}

i = 0
for file in files:
    if file.endswith('.json'):
        # get the corresponding image file
        image_file = file.replace('.json', '.png')
        if not os.path.exists(os.path.join(SRC_FOLDER, image_file)):
            print(f"Image file {image_file} not found for {file}, skipping.")
            continue
        shutil.copy(os.path.join(SRC_FOLDER, image_file), os.path.join(DST_IMAGES_FOLDER, image_file))

        #get the height and width of the image
        image_path = os.path.join(SRC_FOLDER, image_file)
        with Image.open(image_path) as img:
            width, height = img.size
        
        #print(f"Processing {file} with image {image_file} of size {width}x{height}")

        #read the json file and load the data
        with open(os.path.join(SRC_FOLDER, file), 'r') as f:
            data = json.load(f)
        
        output = ''
        for item in data['pieces']:
            #get the class name
            class_name = item['piece']
            #if class_name not in classes:
            #    classes[class_name] = len(classes)
            
            box = item['box']
            output += f"{classes[class_name]} {(box[0] + box[2] / 2)/width} {(box[1] + box[3] / 2)/height} {box[2] / width} {box[3] / height}\n"

        #write the output to a file
        label_file = file.replace('.json', '.txt')
        with open(os.path.join(DST_LABELS_FOLDER, label_file), 'w') as f:
            f.write(output.strip())
        i += 1
        if i >= N:
            break

# generate yaml file for classes
with open('dataset/data.yaml', 'w') as f:
    f.write("path: dataset\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n\n")
    f.write(f"nc: {len(classes)}\n")
    f.write("names: [")
    f.write(', '.join(f'"{name}"' for name in classes.keys()))
    f.write(']\n')



In [22]:
# Chessboard orientation pose
TYPE = 'train'
SRC_FOLDER = 'dataset/' +TYPE
DST_IMAGES_FOLDER = 'dataset/images/' + TYPE
DST_LABELS_FOLDER = 'dataset/labels/' + TYPE
N = 1000  

import os
import shutil
import json
from PIL import Image


#get all files in the source folder
files = os.listdir(SRC_FOLDER)

# emptu the destination folders if they exist
if os.path.exists(DST_IMAGES_FOLDER):
    shutil.rmtree(DST_IMAGES_FOLDER)
if os.path.exists(DST_LABELS_FOLDER):
    shutil.rmtree(DST_LABELS_FOLDER)
# create destination folders
os.makedirs(DST_IMAGES_FOLDER, exist_ok=True)
os.makedirs(DST_LABELS_FOLDER, exist_ok=True)


i = 0
for file in files:
    if file.endswith('.json'):
        # get the corresponding image file
        image_file = file.replace('.json', '.png')
        if not os.path.exists(os.path.join(SRC_FOLDER, image_file)):
            print(f"Image file {image_file} not found for {file}, skipping.")
            continue
        shutil.copy(os.path.join(SRC_FOLDER, image_file), os.path.join(DST_IMAGES_FOLDER, image_file))

        #get the height and width of the image
        image_path = os.path.join(SRC_FOLDER, image_file)
        with Image.open(image_path) as img:
            width, height = img.size
        
        #print(f"Processing {file} with image {image_file} of size {width}x{height}")

        #read the json file and load the data
        with open(os.path.join(SRC_FOLDER, file), 'r') as f:
            data = json.load(f)

            #write the output to a file
            label_file = file.replace('.json', '.txt')
            with open(os.path.join(DST_LABELS_FOLDER, label_file), 'w') as f:
                corners = data['corners']

                # Calculate the axis-aligned bounding box from the corners
                x_coords = [pt[0] for pt in corners]
                y_coords = [pt[1] for pt in corners]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                corners_bbox = [x_min, y_min, x_max - x_min, y_max - y_min]

                #corners = [[100, 500], [500, 500], [500, 100], [100, 100]]  # For testing purposes, replace with actual corners
                
                output = f"0 {(corners_bbox[0] + corners_bbox[2] / 2) / width} {(corners_bbox[1] + corners_bbox[3] / 2) / height} {corners_bbox[2] / width} {corners_bbox[3] / height} {corners[0][0] / width} {corners[0][1] / height} {corners[1][0] / width} {corners[1][1] / height} {corners[2][0] / width} {corners[2][1] / height} {corners[3][0] / width} {corners[3][1] / height}\n"
                #write the output to a file
                f.write(output.strip())
        i += 1
        if i >= N:
            break

# generate yaml file for classes
with open('dataset/data_board_pose.yaml', 'w') as f:
    f.write("path: dataset\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n\n")
    f.write(f"kpt_shape: [4, 2]\n")
    f.write("names:\n    0: board")




In [31]:
# Chessboard orientation segmentation
TYPE = 'val'
SRC_FOLDER = 'dataset/' +TYPE
DST_IMAGES_FOLDER = 'dataset/images/' + TYPE
DST_LABELS_FOLDER = 'dataset/labels/' + TYPE
N = 1000

import os
import shutil
import json
from PIL import Image


#get all files in the source folder
files = os.listdir(SRC_FOLDER)

# emptu the destination folders if they exist
if os.path.exists(DST_IMAGES_FOLDER):
    shutil.rmtree(DST_IMAGES_FOLDER)
if os.path.exists(DST_LABELS_FOLDER):
    shutil.rmtree(DST_LABELS_FOLDER)
# create destination folders
os.makedirs(DST_IMAGES_FOLDER, exist_ok=True)
os.makedirs(DST_LABELS_FOLDER, exist_ok=True)


i = 0
for file in files:
    if file.endswith('.json'):
        # get the corresponding image file
        image_file = file.replace('.json', '.png')
        if not os.path.exists(os.path.join(SRC_FOLDER, image_file)):
            print(f"Image file {image_file} not found for {file}, skipping.")
            continue
        shutil.copy(os.path.join(SRC_FOLDER, image_file), os.path.join(DST_IMAGES_FOLDER, image_file))

        #get the height and width of the image
        image_path = os.path.join(SRC_FOLDER, image_file)
        with Image.open(image_path) as img:
            width, height = img.size
        
        #print(f"Processing {file} with image {image_file} of size {width}x{height}")

        #read the json file and load the data
        with open(os.path.join(SRC_FOLDER, file), 'r') as f:
            data = json.load(f)

            #write the output to a file
            label_file = file.replace('.json', '.txt')
            with open(os.path.join(DST_LABELS_FOLDER, label_file), 'w') as f:
                corners = data['corners']
                #corners = [[100, 500], [500, 500], [500, 100], [100, 100]]  # For testing purposes, replace with actual corners
                
                output = f"0 {corners[0][0] / width} {corners[0][1] / height} {corners[1][0] / width} {corners[1][1] / height} {corners[2][0] / width} {corners[2][1] / height} {corners[3][0] / width} {corners[3][1] / height}\n"
                #write the output to a file
                f.write(output.strip())
        i += 1
        if i >= N:
            break

# generate yaml file for classes
with open('dataset/data_board.yaml', 'w') as f:
    f.write("path: dataset\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n\n")
    f.write(f"nc: 1\n")
    f.write("names: ['board']\n\n")

